In [1]:
import sys
import os
import os.path as osp
import argparse
import numpy as np
import cv2
import json
import torch
from glob import glob
from tqdm import tqdm
import torchvision.transforms as transforms
from torch.nn.parallel.data_parallel import DataParallel
import torch.backends.cudnn as cudnn

sys.path.insert(0, osp.join('..', 'main'))
sys.path.insert(0, osp.join('..', 'data'))
sys.path.insert(0, osp.join('..', 'common'))
from config import cfg
from model import get_model
from utils.preprocessing import load_img, process_bbox, generate_patch_image, get_iou
from utils.vis import vis_keypoints_with_skeleton, save_obj, render_mesh_orthogonal
from utils.mano import mano
from adabn_utils import compute_bn_stats, replace_bn_stats

Fix shapedirs bug of MANO


In [2]:
cfg.set_args("0")
cudnn.benchmark = True

model_path = './snapshot_6.pth'
assert osp.exists(model_path), 'Cannot find model at ' + model_path
print('Load checkpoint from {}'.format(model_path))
model = get_model('test')
model = DataParallel(model).cuda()
ckpt = torch.load(model_path)
model.load_state_dict(ckpt['network'], strict=False)
model.eval()


>>> Using GPU: 0
Load checkpoint from ./snapshot_6.pth


C:\Users\vladi\AppData\Local\Temp\ipykernel_25984\1564869225.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_path)


DataParallel(
  (module): Model(
    (body_backbone): ResNetBackbone(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu

In [3]:
ir_image_dir = r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\IR"
ir_image_paths = glob(osp.join(ir_image_dir, "*.jpg"))  # Assuming images are in .jpg forma

In [4]:
def normalize_coords(x, y, img_width, img_height):
   return {
       "x": float(x) / img_width,
       "y": float(y) / img_height
   }


In [5]:
predictions = []
# Process each image
for img_path in ir_image_paths:
   # Load and process image (your existing code)
   original_img = load_img(img_path)
   img_height, img_width = original_img.shape[:2]
   bbox = [0, 0, img_width, img_height]
   bbox = process_bbox(bbox, img_width, img_height)
   img, img2bb_trans, bb2img_trans = generate_patch_image(original_img, bbox, 1.0, 0.0, False, cfg.input_img_shape)
   transform = transforms.ToTensor()
   img = transform(img.astype(np.float32))/255
   img = img.cuda()[None,:,:,:]
    # Forward pass
   inputs = {'img': img}
   targets = {}
   meta_info = {}
   with torch.no_grad():
       out = model(inputs, targets, meta_info, 'test')
    # Get predictions for both hands
   prediction = {
       "image": os.path.basename(img_path),
       "width": int(img_width),
       "height": int(img_height),
       "landmarks": [],
       "normalized": True
   }
   
   for hand in ('left', 'right'):
       # Get joint coordinates
       joint_img = out[hand[0] + 'joint_img'].cpu().numpy()[0]
       joint_img_xy1 = np.concatenate((joint_img[:,:2], np.ones_like(joint_img[:,:1])),1)
       joint_img = np.dot(bb2img_trans, joint_img_xy1.transpose(1,0)).transpose(1,0)
       
       # Normalize and format coordinates
       hand_points = []
       for point in joint_img:
           hand_points.append(normalize_coords(point[0], point[1], img_width, img_height))
       
       prediction["landmarks"].append(hand_points)
   
   predictions.append(prediction)

c:\Users\vladi\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
base_img_dir = os.path.dirname(ir_image_paths[0])

In [7]:
from PIL import Image

In [8]:
for pred in predictions:
    # Skip if width and height already exist
    if 'width' in pred and 'height' in pred:
        continue
        
    # Get image path
    img_path = os.path.join(base_img_dir, pred['image'])
    print(img_path)
    # Open image and get dimensions
    try:
        with Image.open(img_path) as img:
            width, height = img.size
            # Add dimensions to prediction
            pred['width'] = width
            pred['height'] = height
            pred['normalized'] = True
    except Exception as e:
        print(f"Error processing {pred['image']}: {e}")

In [9]:
output_path = 'predictions_IW_RAW_IR.json'
with open(output_path, 'w') as f:
   json.dump(predictions, f, indent=2)
print(f"Predictions saved to {output_path}")

Predictions saved to predictions_IW_RAW_IR.json


In [10]:
# Load ground truth data
with open(r'C:\Users\vladi\RP\our_hands_dataset_labeled_previews\combined_FIX_RGB.json', 'r') as f:
    ground_truth = json.load(f)
print(f"Loaded {len(ground_truth)} ground truth samples")


Loaded 125 ground truth samples


In [11]:
# Get sets of image names
gt_images = set(sample["image"] for sample in ground_truth)
pred_images = set(sample["image"] for sample in predictions)

print("Images in ground truth but not in predictions:")
print(gt_images - pred_images)
print("\nImages in predictions but not in ground truth:")
print(pred_images - gt_images)
print("\nTotal images in ground truth:", len(gt_images))
print("Total images in predictions:", len(pred_images))

Images in ground truth but not in predictions:
{'53_IMG_4606.jpg', '64_IMG_4617.jpg', '27_IMG_4580.jpg', '26_IMG_4579.jpg', '22_IMG_4575.jpg', '40_IMG_4593.jpg', '21_IMG_4574.jpg', '23_IMG_4576.jpg', '72_IMG_4625.jpg', '69_IMG_4622.jpg', '11_IMG_4564.jpg', '76_IMG_4629.jpg', '74_IMG_4627.jpg', '59_IMG_4612.jpg', '60_IMG_4613.jpg', '55_IMG_4608.jpg', '3_IMG_4556.jpg', '51_IMG_4604.jpg', '67_IMG_4620.jpg', '54_IMG_4607.jpg', '6_IMG_4559.jpg', '24_IMG_4577.jpg', '7_IMG_4560.jpg', '19_IMG_4572.jpg', '42_IMG_4595.jpg', '37_IMG_4590.jpg', '71_IMG_4624.jpg', '15_IMG_4568.jpg', '39_IMG_4592.jpg', '14_IMG_4567.jpg', '4_IMG_4557.jpg', '57_IMG_4610.jpg', '45_IMG_4598.jpg', '47_IMG_4600.jpg', '50_IMG_4603.jpg', '8_IMG_4561.jpg', '18_IMG_4571.jpg', '77_IMG_4630.jpg', '65_IMG_4618.jpg', '36_IMG_4589.jpg', '5_IMG_4558.jpg', '49_IMG_4602.jpg', '70_IMG_4623.jpg', '80_IMG_4633.jpg', '73_IMG_4626.jpg', '33_IMG_4586.jpg', '10_IMG_4563.jpg', '13_IMG_4566.jpg', '63_IMG_4616.jpg', '52_IMG_4605.jpg', '29_IMG_

In [32]:
gt_images == pred_images

True